## Criterio 2 Detección anomalías	
- Nivel 1 (reprobado) No se ve en el código donde se aplique alguna técnica de detección de anomalía individual o grupal.
0 puntos

- Nivel 2 (suficiente) Aunque se codifican algunas técnicas para detectar anomalías a nivel de atributo, no se aplicaron técnicas para detectar anomalías a nivel de instancia.
1 puntos

- Nivel 3 (adecuado) Se aplican algoritmos para la detección de anomalías tanto por atributo como de instancia, pero no se realizan comentarios al respecto.
2 puntos

- Nivel 4 (destacado) Aplica algoritmos para detectar anomalías, tanto a nivel de atributo como de instancia, con muy buenos resultados.
3 puntos

In [18]:
# ISOLATION FOREST
import numpy as np
import pandas as pd

#
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

#
from sklearn.ensemble import IsolationForest
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import plotly.express as px

In [4]:
df_root = pd.read_csv("./CTG.csv")

In [5]:
df_root.head()

,FileName,Date,SegFile,b,e,LBE,LB,AC,FM,UC,...,C,D,E,AD,DE,LD,FS,SUSP,CLASS,NSP
0,Variab10.txt,12/1/1996,CTG0001.txt,240.0,357.0,120.0,120.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0,2.0
1,Fmcs_1.txt,5/3/1996,CTG0002.txt,5.0,632.0,132.0,132.0,4.0,0.0,4.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
2,Fmcs_1.txt,5/3/1996,CTG0003.txt,177.0,779.0,133.0,133.0,2.0,0.0,5.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
3,Fmcs_1.txt,5/3/1996,CTG0004.txt,411.0,1192.0,134.0,134.0,2.0,0.0,6.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
4,Fmcs_1.txt,5/3/1996,CTG0005.txt,533.0,1147.0,132.0,132.0,4.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0


In [13]:
# Eliminar columnas que considero innecesarias
df_cleaned = df_root.drop(columns=["FileName", "SegFile", "Mode","Mean","Median", "Nzeros"])

# Eliminar datos de clasificacion del df
df_cleaned = df_cleaned.drop(columns=["A", "B", "C", "D", "E", "CLASS"])

df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      2126 non-null   object 
 1   b         2126 non-null   float64
 2   e         2126 non-null   float64
 3   LBE       2126 non-null   float64
 4   LB        2126 non-null   float64
 5   AC        2126 non-null   float64
 6   FM        2127 non-null   float64
 7   UC        2127 non-null   float64
 8   ASTV      2127 non-null   float64
 9   MSTV      2127 non-null   float64
 10  ALTV      2127 non-null   float64
 11  MLTV      2127 non-null   float64
 12  DL        2128 non-null   float64
 13  DS        2128 non-null   float64
 14  DP        2128 non-null   float64
 15  DR        2128 non-null   float64
 16  Width     2126 non-null   float64
 17  Min       2126 non-null   float64
 18  Max       2126 non-null   float64
 19  Nmax      2126 non-null   float64
 20  Variance  2126 non-null   floa

In [16]:
df_cleaned.describe()

,b,e,LBE,LB,AC,FM,UC,ASTV,MSTV,ALTV,...,Max,Nmax,Variance,Tendency,AD,DE,LD,FS,SUSP,NSP
count,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2127.000000,2127.000000,2127.000000,2127.000000,2127.000000,...,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000
mean,878.439793,1702.877234,133.303857,133.303857,2.722484,7.503056,3.669017,47.008933,1.335449,9.884814,...,164.025400,4.068203,18.808090,0.320320,0.156162,0.118532,0.050329,0.032455,0.092662,1.304327
std,894.084748,930.919143,9.840844,9.840844,3.560850,39.030452,2.877148,17.210648,0.891543,18.476534,...,17.944183,2.949386,28.977636,0.610829,0.363094,0.323314,0.218675,0.177248,0.290027,0.614377
min,0.000000,287.000000,106.000000,106.000000,0.000000,0.000000,0.000000,12.000000,0.200000,0.000000,...,122.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,55.000000,1009.000000,126.000000,126.000000,0.000000,0.000000,1.000000,32.000000,0.700000,0.000000,...,152.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,538.000000,1241.000000,133.000000,133.000000,1.000000,0.000000,3.000000,49.000000,1.200000,0.000000,...,162.000000,3.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1521.000000,2434.750000,140.000000,140.000000,4.000000,2.000000,5.000000,61.000000,1.700000,11.000000,...,174.000000,6.000000,24.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,3296.000000,3599.000000,160.000000,160.000000,26.000000,564.000000,23.000000,87.000000,7.000000,91.000000,...,238.000000,18.000000,269.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000


In [26]:
#create a histogram
fig = px.histogram(df_cleaned, x = "ASTV")
fig.show()

In [27]:
#create a box plot
fig = px.box(df_cleaned, y="ASTV")
fig.show()

In [28]:
fig = px.scatter(x=df_cleaned['ASTV'], y=df_cleaned['UC'])
fig.show()

In [29]:
#create a function to find outliers using IQR

def find_outliers_IQR(df):
   q1=df.quantile(0.25)
   q3=df.quantile(0.75)
   IQR=q3-q1
   outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]
   return outliers

In [30]:
outliers = find_outliers_IQR(df_cleaned["FM"])
print("number of outliers: "+ str(len(outliers)))
print("max outlier value: "+ str(outliers.max()))
print("min outlier value: "+ str(outliers.min()))
outliers

number of outliers: 311
max outlier value: 564.0
min outlier value: 6.0


12       57.0
13      147.0
14      489.0
15      273.0
16      290.0
        ...  
2086     10.0
2099      7.0
2100      8.0
2101      8.0
2128    564.0
Name: FM, Length: 311, dtype: float64

In [32]:
outliers = find_outliers_IQR(df_cleaned[["UC","FM"]])
outliers

,UC,FM
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
2124,NaN,NaN
2125,NaN,NaN
2126,NaN,NaN
2127,NaN,NaN
